In [1]:
!pip install transformers[torch] datasets -q

In [2]:
import collections

import numpy as np
from datasets import load_dataset
from datasets import load_metric
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import Trainer

In [3]:
datasets = load_dataset("sberquad")

In [4]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 45328
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5036
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 23936
    })
})

In [5]:
print("Context: ", datasets["train"][0]["context"], end='\n\n')
print("Question: ", datasets["train"][0]["question"], end='\n\n')
print("Answer: ", datasets["train"][0]["answers"], end='\n\n')

Context:  В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся скопления графито-углистого вещества, образовавшегося в результате разложения Corycium enigmaticum. В кремнистых сланцах железорудной формации Канады найдены нитевидные водоросли, грибные нити и формы, близкие современным кокколитофоридам. В железистых кварцитах Северной Америки и Сибири обнаружены железистые продукты жизнедеятельности бактерий.

Question:  чем представлены органические остатки?

Answer:  {'text': ['известковыми выделениями сине-зелёных водорослей'], 'answer_start': [109]}


In [6]:
model_name = "DeepPavlov/rubert-base-cased"

model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
max_length = 384
stride = 128


def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [8]:
train_dataset = datasets["train"].select(range(1000)).map(
    preprocess_training_examples,
    batched=True,
    remove_columns=datasets["train"].column_names,
)
len(datasets["train"]), len(train_dataset)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

(45328, 1000)

In [9]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [10]:
validation_dataset = datasets["validation"].select(range(1000)).map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=datasets["validation"].column_names,
)
len(datasets["validation"]), len(validation_dataset)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

(5036, 1002)

In [44]:
metric = load_metric("squad")

n_best = 20
max_answer_length = 30

def compute_metrics(start_logits, end_logits, features, examples):

    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": str(example_id), "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": str(example_id), "prediction_text": ""})

    theoretical_answers = [{"id": str(ex["id"]), "answers": {'text':ex["answers"]['text'],
                                                        'answer_start':ex["answers"]['answer_start']}, } for ex in examples]

    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [51]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    weight_decay=0.01,
    logging_steps=100,
    eval_steps=100
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [52]:
print("Evaluation before training:")
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions

compute_metrics(start_logits, end_logits, validation_dataset, datasets['validation'])

Evaluation before training:


100%|██████████| 5036/5036 [00:02<00:00, 1773.88it/s]


{'exact_match': 9.233518665607624, 'f1': 13.526783186433525}

In [53]:
trainer.train()

Step,Training Loss
100,0.095100
200,0.060800


TrainOutput(global_step=250, training_loss=0.07971783828735352, metrics={'train_runtime': 160.4736, 'train_samples_per_second': 12.463, 'train_steps_per_second': 1.558, 'total_flos': 391945135104000.0, 'train_loss': 0.07971783828735352, 'epoch': 2.0})

In [54]:
print("Evaluation after training:")
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions

compute_metrics(start_logits, end_logits, validation_dataset, datasets['validation'])

Evaluation after training:


100%|██████████| 5036/5036 [00:01<00:00, 2722.38it/s]


{'exact_match': 9.571088165210485, 'f1': 13.783609689905695}